In [1]:
import torch
from trl import SFTTrainer
from datasets import load_dataset
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [1]:
from transformers import TrainingArguments, TextStreamer

In [10]:
!pip install git+https://github.com/huggingface/transformers.git

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-q2arqiw8
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-q2arqiw8
  Resolved https://github.com/huggingface/transformers.git to commit dc8156fdd8af3bc79f98a5af1bcabde4cf131d54
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.46.0.dev0-py3-none-any.whl size=9913899 sha256=135d90b624fe8e5a6d4fbf62d11360e8ccb9ed15a19de46215cb6273471c02b6
  Stored in directory: /tmp/pip-ephem-wheel-cache-xe7ov9g0/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth 2024.9.post4 requires tr

In [2]:
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="meta-llama/Llama-3.2-3B-Instruct",
    max_seq_length=max_seq_length,
    #load_in_4bit=True,
    dtype=None,
)

==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.521 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"], 
    use_rslora=True,
    use_gradient_checkpointing="unsloth"
)

Unsloth 2024.9.post4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [60]:
import torch
from transformers import pipeline

model_id = "meta-llama/Llama-3.2-1B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
messages = [
    {"role": "system", "content": "You are a spelling corrector for the french language. You need to add the accents to the provided sentence. Answer only with the correct spelling of the sentence. Do not remove any words from the sentence."},
    {"role": "user", "content": "Add the accents to the following sentence: Je me suis reveille a 7 heures du matin et j'ai mange un bol de cereales"}
]
outputs = pipe(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'role': 'assistant', 'content': "Je me suis réveillé a 7 heures du matin et j'ai mangé un bol de céréales"}


In [1]:
from datasets import load_dataset

In [2]:
dataset = load_dataset('PleIAs/French-PD-Books', split='train', streaming=True)

Resolving data files:   0%|          | 0/145 [00:00<?, ?it/s]

In [3]:
print(next(iter(dataset)))

{'file_id': 'bpt6k55771383', 'ocr': '100', 'title': 'Des idées napoléoniennes', 'date': '1860', 'author': 'Napoléon III (1808-1873 ; empereur des Français)', 'page_count': 676, 'word_count': 43267, 'character_count': 266537, 'complete_text': '\n \nDES IDEES \nNAPOLÉONIENNES. \n \nPARIS. TYPOGRAPHIE DE HENRI PLON, \nIMPRIMEUR DE L\'EMPEREUR, \n8 , RUE GARANCIERE. \n \n \nNAPOLEON LOUIS BONAPARTE. \n \nDES IDEES \nNAPOLEONIENNES \nPAR LE PRINCE \nNAPOLEON-LOUIS BONAPARTE \nLe vieux système est à bout le nouveau n\'est point assis. (NAPOLEON.) \nPARIS \nHENRI PLON, ÉDITEUR \nR U E G A R A N C E R E , 8 \nAMYOT, ÉDITEUR \nR U E D E LA PAIX, 8 \n1860 \n \n \nL\'IDEE \nNAPOLÉONIENNE. \n" Ce ne sont pas seulement les cendres, » mais les idées de l\'Empereur qu\'il faut " ramener, " \nLondres, 1840. \nDepuis vingt-cinq ans la France s\'épuise en vains efforts pour établir un état de choses durable. Les causes de troubles renaissent sans cesse, et la société ne fait que passer tour à tour d\'un